In [26]:
from bs4 import BeautifulSoup
import requests

In [27]:
response = requests.get("https://en.wikipedia.org/wiki/List_of_sovereign_states")
html_content = response.content
soup = BeautifulSoup(html_content,"html.parser")

In [28]:
table_countries = soup.find("table")

In [29]:
countries_rows = table_countries.find("tbody").find_all("tr")

In [30]:
countries_rows[7].find_all("td")[1].text.replace("A ","").replace("\n","")

'UN member state'

In [31]:
countries_rows[7].find_all("td")[0].find("a")["href"]

'/wiki/Andorra'

In [32]:
country_list = list()
for country_row in countries_rows:
    try:
        columns = country_row.find_all("td")
        if columns[1].text.replace("A ","").replace("\n","") == 'UN member state':
            country_a = columns[0].find("a")
            country_element = {"title":country_a["title"],"link":country_a["href"]}
            country_list.append(country_element)
    except:
        pass

In [37]:
country = country_list[0]
response = requests.get("https://en.wikipedia.org" + country.["link"])
content = response.content
soup_afgh = BeautifulSoup(content)

In [45]:
div_content_element = soup_afgh.find("div",{"id":"mw-content-text"}).find(class_="mw-parser-output")

In [53]:
start = False
text = country["title"] + "\n"
for element in div_content_element:
    if start == True and element.name == "p":
        text += element.text
    elif element.name == "table":
        start = True
    elif element.name == "metadata":
        break
    
        

In [55]:
with open(country["title"]+".txt","w",encoding="utf8") as f:
    f.write(text)

UnicodeEncodeError: 'charmap' codec can't encode character '\u0101' in position 5608: character maps to <undefined>